# Establishing a Connection - Carol
## Role: Inviter

In this notebook we'll be initiating the aries DID Exchange protocol. Before an agent can begin to interact with another agent they must engage in this protocol resulting in the exchange of Decentralised Identifiers (DIDs) and corresponding DID Documents. The information within these documents can then be used to establish a secure communication channel between the parties.

For a detailed description of this protocol it is highly recommened to read this [aries-rfc]((https://github.com/hyperledger/aries-rfcs/tree/master/features/0023-did-exchange).

This notebook has the following phases:

1. Pull in dependencies
2. Instantiate the controller for the aries agent (See the docker-compose.yml)
3. Set up a listener for basicmessages events emitted by the controller when it receives a webhook
4. Use the controller to create an invitation from our agent
5. Copy the invitation output from 4 and move over to Bob's notebook

<b>Carry on in [Bob's notebook](http://localhost:8889/lab/tree/1%20Basic%20Concepts%20and%201st%20Connection/Part%203%20-%20Establishing%20a%20Connection.ipynb)</b>
    
11. Accept Request for Connection
12. Send Trust Ping to Bob

    



### 1. Pull in dependencies

In [11]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


### 2. Instatiate the controller for our Agent

The arguments depend on how the aca-py agent was initiated. See the manage and docker-compose.yml files for more details.

In [12]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8082
WEBHOOK_BASE = ""
ADMIN_URL = "http://carol-agent:8081"

# WARNING: You should use environment variables for this
# TODO: Make env variables accessible through juypter notebooks
API_KEY = "carol_api_123456789"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(admin_url=ADMIN_URL, api_key=API_KEY)


In [13]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST, 
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

### 3. Listen for webhooks and register default listeners

Everytime a webhook is received from the agent, the controller reemits the hook using [PyPubSub](https://pypubsub.readthedocs.io/en/v4.0.3/). The default listeners are used to update state and print logs.

In [14]:

loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def connection_handler(payload):
    print("Connection Handler Called")
    connection_id = payload["connection_id"]
    state = payload["state"]
    print(f"Connection {connection_id} in State {state}")
    
connection_listener = {
    "handler": connection_handler,
    "topic": "connections"
}

agent_controller.register_listeners([connection_listener], defaults=True)

Subscribing too: connections


### 4. Use the controller to create an invitation from our agent

An invitation is a JSON object, as shown below, it contains the relevant information required for another agent to connect with it and exchange identifiers. This object must be passed to the agent Alice wishes to connect to out of band, in this instance we will just copy it across.

In [15]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
invite_message = invite['invitation']
print("Connection ID", connection_id)
print("Invitation")
print(invite_message)

Connection ID 1c65e6ce-773a-4d92-bbd8-a966b1531b27
Invitation
{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '94a6bb1b-c2bc-431f-98d4-a83e0b3e2aa5', 'recipientKeys': ['5NqLisZNrS2AbeBNCQHcF7hZx22Xd6aYx8vxadbdkqmJ'], 'label': 'Carol', 'serviceEndpoint': 'http://192.168.65.3:8080'}


### 5. Copy the invitation output from 4 and move over to the [Bob notebook](http://localhost:8889/lab/tree/1%20Basic%20Concepts%20and%201st%20Connection/Part%203%20-%20Establishing%20a%20Connection.ipynb) 

### 11. Accept Request for Connection

In [16]:
# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(connection_id)
print("ACCEPT REQUEST")
print(connection)
print("state", connection["state"])

ACCEPT REQUEST
{'invitation_key': '5NqLisZNrS2AbeBNCQHcF7hZx22Xd6aYx8vxadbdkqmJ', 'their_label': 'Bob', 'state': 'response', 'created_at': '2023-01-02 08:51:39.261613Z', 'their_did': '3jAoJUdyYtxQa6AkTbaeAm', 'connection_id': '1c65e6ce-773a-4d92-bbd8-a966b1531b27', 'routing_state': 'none', 'rfc23_state': 'response-sent', 'my_did': 'QWLH91NGnms8E57znbMPbX', 'updated_at': '2023-01-02 08:52:18.508018Z', 'accept': 'manual', 'invitation_mode': 'once', 'their_role': 'invitee'}
state response


### 12. Send Trust Ping to activate the conneciton

In [17]:
trust_ping = await agent_controller.messaging.trust_ping(connection_id, "hello")
print("Trust Ping", trust_ping)

Trust Ping {'thread_id': 'e9d2eb50-7a59-4fa5-9536-0ad3eb0746a0'}


####  12.1 Check if connection is active

In [18]:
connection = await agent_controller.connections.get_connection(connection_id)
print(connection)
print("Is Active?", connection["state"])

{'invitation_key': '5NqLisZNrS2AbeBNCQHcF7hZx22Xd6aYx8vxadbdkqmJ', 'their_label': 'Bob', 'state': 'active', 'created_at': '2023-01-02 08:51:39.261613Z', 'their_did': '3jAoJUdyYtxQa6AkTbaeAm', 'connection_id': '1c65e6ce-773a-4d92-bbd8-a966b1531b27', 'routing_state': 'none', 'rfc23_state': 'completed', 'my_did': 'QWLH91NGnms8E57znbMPbX', 'updated_at': '2023-01-02 08:52:22.390542Z', 'accept': 'manual', 'invitation_mode': 'once', 'their_role': 'invitee'}
Is Active? active


## End of Tutorial

#### Terminate Controller & Stop Webhook Server

**Note: You will need to run this command when combining this example with others such as Issuer**

In [19]:
response = await agent_controller.terminate()
print(response)

None
